# Opérateurs de mis-à-jour
Les operateurs de mis-à-jour peuvent etre utilisé avec les fonctions:
- update_one()
- update_many()
- find_one_and_update()

In [30]:
from pymongo import MongoClient
from pandas import DataFrame as df

def connect_to_db(url=None):
    return MongoClient(url)

connection = connect_to_db("localhost:27017")
db = connection["update_operation"]

def display_collection(collection):
    display(df(collection).drop("_id", axis=1))

## `$set`

Modifie un dictionnaire avec la valeur associé.   
Si certains champs n'existe pas ils seront crées.  

In [ ]:
collection = db["set"]
collection.insert_one({"id": 30, "dessert": "Glace à la vanille"})

print("Notre glace :")
display_collection(collection.find())

# on change la description du dessert pour une mousse
collection.update_one({"id": 30}, {"$set": {"dessert": "Smoothie"}})
print("Notre glace est rester trop longtemps au soleil :")
display_collection(collection.find())

# $set va aussi créer les champs si besoin
collection.update_one(
    {"id": 30}, {"$set": {"accompagnement": "Copeaux de chocolat"}}
)
print("On en profite pour créer un nouveau dessert :")
display_collection(collection.find())

Notre glace :


,id,dessert
0,30,Glace à la vanille


Notre glace est drester trop longtemps au soleil :


,id,dessert
0,30,Smoothie


On en profite pour créer un nouveau dessert :


,id,dessert,accompagnement
0,30,Smoothie,Copeaux de chocolat


## `$unset`
Supprime un champs s'il existe.

In [ ]:
collection = db["unset"]
collection.insert_one({"personnage": "superman", "accessoire": "Cape rouge"})
display_collection(collection.find())

# on supprime le champs 'accessoire'
collection.update_one({"personnage": "superman"}, {"$unset": {"accessoire": ""}})
print("Oh non, Superman à perdu ça cape :")
display_collection(collection.find())

Oh non, Superman à perdu ça cape :


,personnage
0,superman


## `$inc`
Incremente un champ avec la valeur spécifié.  
Si un champs n'existe pas, on le créer en lui assignement la valeur spécifié.  
C'est une opération atomique


In [ ]:
collection = db['inc']
collection.insert_one({'id': 0, 'cookies': 10, 'Coca': 3, 'Pepsi': 1})
print("Nos stocks :")
display_collection(collection.find())

collection.update_one({'id': 0}, {'$inc': {'cookies': -2, 'Coca': -1}})
print("Des clients sont passés :")
display_collection(collection.find())

collection.update_one({'id': 0}, {'$inc': {'cookies': 2, 'Coca': 8, 'Pepsi': 9, 'Croissant': 10}})
print("On a reçu une livraison :")
display_collection(collection.find())

Des clients sont passés :


,id,cookies,Coca,Pepsi
0,0,8,2,1


On a reçu une livraison :


,id,cookies,Coca,Pepsi,Croissant
0,0,10,10,10,10


## `$mul`

Meme chose que `$inc` mais avec une multiplication au lieu d'une addition.  
Si le champs n'existe pas, il est crée et sa valeur est 0 (on conserve le type).  
C'est une opération atomique
  
/!\ lors de multiplications entre différents types. Des conversions automatiques peuvent etre réalisés. Se référer à la documentation [mongodb](https://www.mongodb.com/docs/manual/reference/operator/update/mul/)

In [34]:
# on reprend intentionellement la collection de $inc
collection.update_one({'id': 0}, {'$mul': {'cookies': 3}})
print('On a commandé 3 fois trop de cookies. Mince alors !')
display_collection(collection.find())

On a commandé 3 fois trop de cookies. Mince alors !


,id,cookies,Coca,Pepsi,Croissant
0,0,30,10,10,10


## `$rename`

Renomme un Champs

Opération non atomique (Delete + $set)

In [35]:
collection = db["rename"]

collection.insert_one({"id": 2, "ortaugrafe": False})
print("Nooon, des fautes d'orthographes :")
display_collection(collection.find())

collection.update_one({"id": 2}, {"$rename": {"ortaugrafe": "orthographe"}})
print("C'est bon, on est sauf :")
display_collection(collection.find())


Nooon, des fautes d'orthographes :


,id,ortaugrafe
0,2,False


C'est bon, on est sauf :


,id,orthographe
0,2,False


## `$push`
Ajoute un élément à un Array.
Si le champs n'existe pas. Il est crée avec l'élément à l'intérieur de l'array.
Echoue si le champs n'est pas un Array.


In [ ]:
collection = db["push"]
collection.insert_one({'id': 3, 'tableaux': ['Le Radeau de la Méduse']})
print("Ma collection :")
display_collection(collection.find())

collection.update_one({'id': 3}, {'$push': {'tableaux': "Knight at the Crossroads"}})
print("Une nouvelle pièce dans ma collection :")
display_collection(collection.find())

collection.update_one({'id': 3}, {'$push': {'gravures': 'Lucifer, King of Hell'}})
print("Ah, maintenant je sais ca que je regarde !")
display_collection(collection.find())

Ma collection :


,id,tableaux
0,3,"[Le Radeau de la Méduse, Knight at the Crossro..."


Une nouvelle pièce dans ma collection :


,id,tableaux
0,3,"[Le Radeau de la Méduse, Knight at the Crossro..."


Ah, maintenant je sais ca que je regarde !


,id,tableaux,gravures
0,3,"[Le Radeau de la Méduse, Knight at the Crossro...","[Lucifer, King of Hell]"


## `$pull`
Retire d'un Array tous les instances de la valeur spécifié.


In [ ]:
collection = db['pull']
collection.insert_one({'id': 1, 'fruits': ['pomme', 'kiwi', 'banane', 'fraise', 'kiwi']})
display_collection(collection.find())

collection.update_one({'id': 1}, {'$pull': {"fruits": 'kiwi'}})
print("les kiwis c'est trop bon! on a tous mangé !")
display_collection(collection.find())

les kiwis c'est trop bon! on a tous mangé !


,id,fruits
0,1,"[pomme, banane, fraise]"


In [39]:
connection.close()